## Check and clean solar farm geocoding

Sara Johns

sjohns@berkeley.edu

11/11/21

### Set up

In [ ]:
# Import libraries
import pandas as pd
import numpy as np

# Data directory
data_path = '/Users/sarajohns/Google Drive/My Drive/Farmers_Solar/data/ipa_lottery/processed/'

# Read in geocoded data
grA_geo = pd.read_csv(data_path + 'group_a_addresses_geocoded_manual_updates.csv')
grB_geo = pd.read_csv(data_path + 'group_b_addresses_geocoded_manual_updates.csv')

# Read in pre-geocoded info
grA_pregeo = pd.read_csv(data_path + 'group_a_addresses.csv')
grB_pregeo = pd.read_csv(data_path + 'group_b_addresses.csv')

### Merge geocoded data with other info

In [ ]:
# Rows numbers are the same so just combine columns
grA = pd.concat([grA_geo.reset_index(drop=True), grA_pregeo], axis=1)
grB = pd.concat([grB_geo.reset_index(drop=True), grB_pregeo], axis=1)

# Check that rows are matched correctly
grA['input_string'].equals(grA['clean_add'])

In [ ]:
grB['input_string'].equals(grB['clean_add'])

### Examine accuracy of geocoding

In [ ]:
# How many failed? Group A
pd.crosstab(index = grA['status'], columns = 'count')

In [ ]:
# How many failed? Group B
pd.crosstab(index = grB['status'], columns = 'count') 

Not many failed, so we can try to manually collect those. (Sometimes these are parcel pins that we can look up).

In [ ]:
# How accurate? Group A
pd.crosstab(index = grA['accuracy'], columns = 'count') 

In [ ]:
# How accurate? Group B
pd.crosstab(index = grB['accuracy'], columns = 'count')

Range interpolated and approximate are not very accurate. So we went back to manually check those. Some were parcel pins we could look up, some were addresses that needed slight updating for Google to find the right location. But others were only streets that map to many farms. These we cannot use.

In [ ]:
# Combine group A and group B in to one table
grA['group'] = "A"
grB['group'] = "B"

all_loc = pd.concat([grA.reset_index(drop=True), grB])

In [ ]:
# Combine manually updated lat/lon and google geocoded lat/lon
all_loc['lat'] = all_loc['man_latitude']
all_loc['lat'] = all_loc['lat'].fillna(all_loc['latitude'])

all_loc['lon'] = all_loc['man_longitude']
all_loc['lon'] = all_loc['lon'].fillna(all_loc['longitude'])

# Replace lat/lon as NA if maps to many
all_loc['lat'] = all_loc.apply(lambda x: None if x['check_note']=="maps_to_many" else x['lat'], axis=1)
all_loc['lon'] = all_loc.apply(lambda x: None if x['check_note']=="maps_to_many" else x['lon'], axis=1)

# Make indicator for manually updated
all_loc['man_update'] = 1 - all_loc['man_latitude'].isna().astype(int)
all_loc['man_update'] = all_loc.apply(lambda x: 1 if x['check_note']=="maps_to_many" else x['man_update'], axis=1)

# Select columns we want
all_loc = all_loc[['Application ID',
                   'Project Name',
                   'Name of Approved Vendor',
                   'group',
                   'lat',
                   'lon',
                   'Size (kW AC)',
                   'Address',
                   'Ordinal',
                   'Lottery Status',
                   'Block',
                   'Queue Number',
                   'Small Subscribers',
                   'formatted_address', 
                   'accuracy', 
                   'type',  
                   'clean_add',
                   'man_update']]

Now we need to bring in the lottery applicants that listed a lat/lon rather than an address.

In [ ]:
# Read in projects that listed coordinates instead of addresses
grA_pt = pd.read_csv(data_path + 'group_a_coordinates.csv')
grB_pt = pd.read_csv(data_path + 'group_b_coordinates.csv')

# Add group identifier
grA_pt['group'] = "A"
grB_pt['group'] = "B"

In [ ]:
# Append points to other data
all_loc = all_loc.append(pd.DataFrame(data = grA_pt), ignore_index=True)
all_loc = all_loc.append(pd.DataFrame(data = grB_pt), ignore_index=True)

In [ ]:
# Save final dataset
all_loc.to_csv(data_path + 'final_lottery_locations.csv')